In [1]:
%pip install pandas numpy matplotlib seaborn statsmodels requests openpyxl yfinance kaggle python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install /home/pinagm/beaapi-0.0.2-py3-none-any.whl

## Getting the ETF Fund data from State Street Global Advisors

In [ ]:
import requests as req
import pandas as pd
import os
import subprocess
import zipfile
import  yfinance as yf
from typing import List
import beaapi as bea
from dotenv import load_dotenv
from datetime import datetime
# Output the filtered DataFrame
load_dotenv()


In [ ]:
def fetch_and_process_etf_data(etf_list):
    ssga = [etf.lower() for etf in etf_list]
    subfolder = 'SSGA Data'
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)

    ssga_df_list = []

    for etf in ssga:
        url = f'https://www.ssga.com/library-content/products/fund-data/etfs/us/holdings-daily-us-en-{etf}.xlsx'
        response = req.get(url)
        current_date = datetime.now().strftime('%m-%d-%Y')
        file_path = os.path.join(subfolder, f'{etf}-{current_date}.xlsx')
        with open(file_path, 'wb') as file:
            file.write(response.content)
        df = pd.read_excel(file_path, skiprows=4, header=0, usecols="A:H")
        
        drop_index = df[df['Name'] == "Past performance is not a reliable indicator of future performance. Investment return and principal value will fluctuate, so you may have a gain or loss when shares are sold. Current performance may be higher or lower than that quoted. All results are historical and assume the reinvestment of dividends and capital gains. Visit www.ssga.com for most recent month-end performance. "].index

        if not drop_index.empty:
            df = df[:drop_index[0]]

        if df.iloc[-1].isna().all():
            df = df[:-1]

        ssga_df_list.append(df)

    return ssga_df_list

# Testing Output
etf_list = ['XLI', 'XLK', 'XLE', 'XLB']
ssga_df_list = fetch_and_process_etf_data(etf_list)
for df in ssga_df_list:
    print(df.tail())

In [ ]:
def fetch_bea_data(api_key, years):
    # base_url = "https://apps.bea.gov/api/data/"
    # params = {
    #     "UserID": api_key,
    #     "method": "GetData",
    #     "datasetname": "GDPbyIndustry",
    #     "Frequency": "A,Q",
    #     "Year": years,
    #     "Industry": "ALL",
    #     "TableID": "ALL",
    #     "ResultFormat": "JSON"
    # }
    try:
        # response = req.get(base_
        # 
        # 
        # url, params=params)
        # response.raise_for_status()
        tbl = bea.get_data(api_key, datasetname="GDPbyIndustry", Frequency="A,Q", Year="2020,2021,2022,2023,2024", TableID="ALL", Industry="ALL")
        bea_df_new = pd.DataFrame(bea.to_wide_vars_in_rows(tbl))
        data = bea_df_new
        # if 'Error' in data['BEAAPI']:
        #     error_code = data['BEAAPI']['Error']['APIErrorCode']
        #     error_description = data['BEAAPI']['Er4ror']['APIErrorDescription']
        #     print(f"API request failed with error4 code {error_code}: {error_description}")
        #     return None
        print(data)
        return data
    except Exception as e:
        print(f"API request failed: {e}")
        return None

def process_bea_data(data):
    # if 'BEAAPI' not in data or 'Results' not in data['BEAAPI']:
    #     print("Error: 'Results' key not found in the API response.")
    #     return None
    # results = data['BEAAPI']['Results']
    
    bea_df = pd.DataFrame(data)
    bea_df.rename(columns={"IndustrYDescription": "IndustryDescription"}, inplace=True)
    grouped_df = bea_df.groupby(['Industry', 'IndustryDescription'], as_index=False)['DataValue'].sum()
    filtered_df = grouped_df[~grouped_df['IndustryDescription'].isin([
        "Taxes on production and imports less subsidies",
        "Compensation of employees",
        "Gross operating surplus"
    ])]
    return filtered_df

def save_bea_data(filtered_df):
    subfolder = 'BEA Data'
    os.makedirs(subfolder, exist_ok=True)
    current_date = datetime.now().strftime('%m-%d-%Y')
    file_path = os.path.join(subfolder, f"bea-gdp-by-industry-raw-{current_date}.xlsx")
    filtered_df.to_excel(file_path, index=False)
    return file_path

def map_sectors(filtered_df):
    sector_map = {
        "Technology": [
            "Computer and electronic products",
            "Computer systems design and related services",
            "Data processing, internet publishing, and other information services",
            "Information-communications-technology-producing industries"
        ],
        "Materials": [
            "Agriculture, forestry, fishing, and hunting",
            "Farms",
            "Forestry, fishing, and related activities",
            "Mining",
            "Mining, except oil and gas",
            "Support activities for mining",
            "Wood products",
            "Paper products",
            "Chemical products",
            "Plastics and rubber products",
            "Nonmetallic mineral products",
            "Primary metals",
            "Fabricated metal products"
        ],
        "Energy": [
            "Oil and gas extraction",
            "Petroleum and coal products",
            "Pipeline transportation"
        ],
        "Industrials": [
            "Construction",
            "Machinery",
            "Electrical equipment, appliances, and components",
            "Other transportation equipment",
            "Miscellaneous manufacturing",
            "Durable goods",
            "Wholesale trade",
            "Rail transportation",
            "Water transportation",
            "Truck transportation",
            "Transit and ground passenger transportation",
            "Other transportation and support activities",
            "Transportation and warehousing",
            "Warehousing and storage",
            "Waste management and remediation services"
        ]
    }

    def get_sector(category):
        for sector, cat_list in sector_map.items():
            if category in cat_list:
                return sector
        return "Other"

    filtered_df = filtered_df.copy()
    filtered_df.loc[:, "Sector"] = filtered_df["IndustryDescription"].apply(get_sector)
    focus_sectors = ["Technology", "Materials", "Energy", "Industrials"]
    df_filtered = filtered_df[filtered_df["Sector"].isin(focus_sectors)]
    return df_filtered

def fetch_bea_gdp_by_industry(api_key, years):
    data = fetch_bea_data(api_key, years)
    if data is None:
        return None
    filtered_df = process_bea_data(data)
    if filtered_df is not None:
        save_bea_data(filtered_df)
        df_filtered = map_sectors(filtered_df)
        current_date = datetime.now().strftime('%m-%d-%Y')
        subfolder = 'BEA Data'
        filtered_file_path = os.path.join(subfolder, f"bea-gdp-by-industry-filtered-{current_date}.xlsx")
        df_filtered.to_excel(filtered_file_path, index=False)
        return df_filtered
    else:
        print("Error: Processed data is None.")
        return None

# Output the filtered DataFrame
api_key = os.environ.get("beakey")
years = "2020,2021,2022,2023, 2024"
bea_df = fetch_bea_gdp_by_industry(api_key, years)
if bea_df is not None:
    print(bea_df.tail())

In [ ]:
def setup_kaggle_data():
    # Step 1: Ensure the "Kaggle Data" directory exists
    kaggle_data_dir = "Kaggle Data"
    os.makedirs(kaggle_data_dir, exist_ok=True)
    
    # Step 2: Check if the .csv file already exists
    csv_exists = any(filename.endswith('.csv') for filename in os.listdir(kaggle_data_dir))
    
    if not csv_exists:
        # Step 3: Download the dataset using the Kaggle CLI
        dataset = "jakewright/9000-tickers-of-stock-market-data-full-history"
        subprocess.run(["kaggle", "datasets", "download", "-d", dataset], check=True)
        
        # Step 4: Unzip the downloaded file
        zip_filename = dataset.split('/')[-1] + ".zip"
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(kaggle_data_dir)
        
        # Step 5: Remove any files in the "Kaggle Data" directory that are not .csv files
        for filename in os.listdir(kaggle_data_dir):
            if not filename.endswith('.csv'):
                os.remove(os.path.join(kaggle_data_dir, filename))
        
        # Optionally, remove the downloaded zip file
        os.remove(zip_filename)

# Call the function
setup_kaggle_data()

# Load the CSV file into a DataFrame
kag_df = pd.read_csv('Kaggle Data/all_stock_data.csv')

# Convert the 'Date' column to datetime format
kag_df['Date'] = pd.to_datetime(kag_df['Date'])

# Filter the DataFrame for dates between 2020-01-01 and 2023-12-31
start_date = '2020-01-01'
end_date = '2023-12-31'
filtered_kag_df = kag_df[(kag_df['Date'] >= start_date) & (kag_df['Date'] <= end_date)]

# Display the first few rows of the filtered DataFrame
filtered_kag_df.head()

In [ ]:
filtered_kag_df.tail()

In [ ]:
def ensure_directory_exists(directory: str) -> None:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists.")

def download_data(tickers: List[str], start_date: str, end_date: str, interval: str) -> pd.DataFrame:
    return yf.download(
        tickers=tickers,
        start=start_date,
        end=end_date,
        interval=interval,
        group_by="ticker",
        auto_adjust=True,
        threads=True,
        progress=True
    )

def filter_columns(df: pd.DataFrame, tickers: List[str], fields: List[str]) -> pd.DataFrame:
    keep_cols = [(t, field) for t in tickers for field in fields if (t, field) in df.columns]
    return df[keep_cols].copy()

def add_change_columns(df: pd.DataFrame, tickers: List[str]) -> pd.DataFrame:
    for t in tickers:
        if (t, "High") in df.columns and (t, "Low") in df.columns:
            df[(t, "Change")] = df[(t, "High")] - df[(t, "Low")]
    return df

def process_change_columns(df: pd.DataFrame) -> pd.DataFrame:
    df_only_change = df.loc[:, (slice(None), "Change")].copy()
    df_only_change.columns = df_only_change.columns.droplevel(1)
    df_only_change.columns = [f"{ticker}_Change" for ticker in df_only_change.columns]
    df_only_change.reset_index(inplace=True)
    if df_only_change.columns[0] == "index":
        df_only_change.rename(columns={"index": "Date"}, inplace=True)
    return df_only_change

def save_to_csv(df: pd.DataFrame, path: str) -> None:
    df.to_csv(path, index=False)
    print(f"Data with only 'Change' columns saved to: {path}")

# Main execution flow
data_dir = "Yahoo Data"
ensure_directory_exists(data_dir)

tickers = ["XLI", "XLK", "XLE", "XLB"]
df_full = download_data(tickers, "2020-01-01", "2024-01-01", "3mo")

df_filtered = filter_columns(df_full, tickers, ["High", "Low"])
df_filtered = add_change_columns(df_filtered)

df_only_change = process_change_columns(df_filtered)

csv_path = os.path.join(data_dir, "sector_quarterly_only_change.csv")
save_to_csv(df_only_change, csv_path)

print(df_only_change.head())